In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import seaborn as  sns
import matplotlib.pyplot as plt
import gc
import matplotlib as mpl
from matplotlib.pyplot import savefig

mpl.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题

%matplotlib inline

## 订单数据整合
### 常用字段

In [2]:
cn_short_col = ['订单号','保单号','订单状态','下单日期','支付日期','承保时间','支付方式','保单金额','理赔津贴-购买金额',
                '渠道来源','来源明细','推广费比例A','推广费比例B','推广费A','推广费B','推广费税费','推荐费税费','推荐费',
                '推广费b1','推广费b2','红包金额',
                '起保时间','终保时间','商品编码','产品ID','产品名称','保险公司','批量流水号','计划名称',
                '订单续期来源','是否自保件',
                '服务网点ID','销售团队名称','销服id','销服名称','营业部ID','营业部名称',
                '福利券编码','福利券名称','福利券比例','上限金额','福利固定面值','推广费X',
                '创建人ID','推广费代理人','推广费代理人ID','推广费代理人电话',
                '推广费代理人省','推广费代理人市','推广费代理人区','推广费代理人展业地点','推广费代理人服务网点',
                '是否司内用户','核心代理人编码','核心入职时间','区域中心',
                '推荐费代理人姓名','推荐费代理人ID','推荐费代理人电话','推荐费代理人省','推荐费代理人市','推荐费代理人区',
                '推荐费代理人展业地点','推荐费代理人服务网点',
                '投保人','投保人证件类型','投保人证件号','投保人电子邮箱','投保人电话',
                '与投保人关系','被保人姓名','被保人证件类型','被保人证件号']
en_short_col = ['order','order_origin','order_stat','time_order','time_pay','time_cb','pay_way','bf','bf_add',
               'channel_src','channel_detail','tgf_rate_A','tgf_rate_B','tgf_A','tgf_B','tgf_tax','tjf_tax','tjf',
                'tgf_b1','tgf_b2','money_add',
                'time_start','time_end','pcode','pid','pname','pcompany','batch_id','plan',
                'order_renew_src','is_self',
                'area_service_id','team_leader','team_sale_id','team_sale_name','team_bus_id','team_bus_name',
                'ticket_code','ticket_name','ticket_rate','money_top','ticket_value','tgf_X',
                'order_origin_uid','uname','uid','ucell',
                'uprov','ucity','uarea','area_work','area_service',
                'is_insider','ucode','time_join','area_type',
                'tj_name','tj_uid','tj_cell','tj_prov','tj_city','tj_area','tj_area_work','tj_area_service',
                'tb_name','tb_id_type','tb_id','tb_email','tb_cell',
                'relation','bb_name','bb_id_type','bb_id']
order_short_col = dict(zip(cn_short_col,en_short_col))

In [3]:
cn_life_col = [
                '订单号','保单号','订单状态','下单日期','支付日期','承保时间','支付方式','保单金额',
                '渠道来源','来源明细','推广费比例A','推广费比例B','推广费A','推广费B','推广费税费','推荐费税费','推荐费',
                '起保时间','商品编码','产品ID','产品名称','保险公司','合并订单流水号','缴费期间','是否自保件',
                '福利券编码','福利券名称','福利券比例','上限金额','福利固定面值','推广费X',
                '创建人ID','推广费代理人','推广费代理人ID','推广费代理人电话',
                '推广费代理人省','推广费代理人市','推广费代理人区','推广费代理人展业地点','推广费代理人服务网点',
                '是否司内用户','核心代理人编码','核心入职时间','区域中心',
                '推荐费代理人姓名','推荐费代理人ID','推荐费代理人电话','推荐费代理人省','推荐费代理人市','推荐费代理人区',
                '推荐费代理人展业地点','推荐费代理人服务网点',
                '投保人','投保人证件类型','投保人证件号','投保人电子邮箱','投保人电话',
                '与投保人关系','被保人姓名','被保人证件类型','被保人证件号',
                '服务网点ID','销售团队名称','销服id','销服名称','营业部ID','营业部名称',
                '红包金额','推广费b1','推广费b2']
en_life_col = ['order','order_origin','order_stat','time_order','time_pay','time_cb','pay_way','bf',
               'channel_src','channel_detail','tgf_rate_A','tgf_rate_B','tgf_A','tgf_B','tgf_tax','tjf_tax','tjf',
                'time_start','pcode','pid','pname','pcompany','batch_id','plan','is_self',
                'ticket_code','ticket_name','ticket_rate','money_top','ticket_value','tgf_X',
                'order_origin_uid','uname','uid','ucell',
                'uprov','ucity','uarea','area_work','area_service',
                'is_insider','ucode','time_join','area_type',
                'tj_name','tj_uid','tj_cell','tj_prov','tj_city','tj_area','tj_area_work','tj_area_service',
                'tb_name','tb_id_type','tb_id','tb_email','tb_cell',
                'relation','bb_name','bb_id_type','bb_id',
                'area_service_id','team_leader','team_sale_id','team_sale_name','team_bus_id','team_bus_name',
                'money_add','tgf_b1','tgf_b2']
order_life_col = dict(zip(cn_life_col,en_life_col))

In [4]:
dir_dic   = 'E:\\SynologyDrive\\dir_dic\\'
dir_login = 'E:\\SynologyDrive\\dir_login\\'
dir_order = 'E:\\SynologyDrive\\dir_order\\'
dir_user  = 'E:\\SynologyDrive\\dir_user\\'
dir_order_life = 'E:\\SynologyDrive\\dir_order\\order_life\\'
dir_score = 'E:\\SynologyDrive\\dir_score\\'
dir_sup = 'E:\\SynologyDrive\\data_support_pro\\'
dir_report = 'E:\\SynologyDrive\\日报明细数据\\'
dir_download = 'D:\\User\\Downloads\\'

date_bus = pd.read_excel(dir_dic  + '03_time_index.xlsx',parse_dates=['date_pay'])
d1= list(date_bus.date_pay)
d2 = list(date_bus.mon_business)
date_bus = dict(zip(d1,d2))

### 今年累积订单

In [5]:
## 修改-日期
order_2020 = pd.read_csv('D:\\User\\Desktop\\order_info\\order_detail_2020_0521.csv',encoding='utf-8',parse_dates=['date_pay'],low_memory=False)
order_2020.groupby('date_pay').bf.sum().reset_index().tail()

,date_pay,bf
143,2020-05-17,1106044.36
144,2020-05-18,1470095.30
145,2020-05-19,1485886.14
146,2020-05-20,1874182.58
147,2020-05-21,1415569.52


### 今日订单

In [6]:
order_file_old_1  = dir_download + '非车险订单管理1.xlsx'
order_file_old_2 = dir_download + '定期寿订单管理.xlsx'
order_file_new_1 = dir_order + '非车险订单管理.xlsx'
order_file_new_2 = dir_order + '定期寿订单管理.xlsx'

shutil.copyfile(order_file_old_1,order_file_new_1)
print('1_finish')
shutil.copyfile(order_file_old_2,order_file_new_2)
print('2_finish')

1_finish
2_finish


In [7]:
order_short_day = pd.read_excel(order_file_new_1,parse_dates=['支付日期'])
order_life_day = pd.read_excel(order_file_new_2,parse_dates=['支付日期'])
order_short_day.rename(columns=order_short_col,inplace=True)
order_life_day.rename(columns=order_life_col,inplace=True)
order_life_day['time_end'] = np.nan
order_life_day['order_renew_src'] = np.nan
order_short_day['date_pay'] = pd.to_datetime(order_short_day.time_pay.dt.strftime('%Y-%m-%d'))
order_life_day['date_pay'] = pd.to_datetime(order_life_day.time_pay.dt.strftime('%Y-%m-%d'))

### 数据整合

In [8]:
en_short_col = ['order','order_origin','order_stat','time_order','time_pay','time_cb',
                'time_start','time_end','pcode','pid','pname','pcompany','plan',
                'pay_way','channel_src','channel_detail','batch_id','order_renew_src','is_self',
                'bf','tgf_rate_A','tgf_rate_B','tgf_A','tgf_B','tgf_tax','tjf_tax','tjf','tgf_b1','tgf_b2','money_add',
                'ticket_code','ticket_name','ticket_rate','money_top','ticket_value','tgf_X',
                'uid','order_origin_uid','uname','ucell','uprov','ucity','uarea',
                'is_insider','ucode','time_join','area_type','area_work',
                'area_service_id','area_service','team_sale_id','team_sale_name','team_bus_id','team_bus_name',
                'tj_name','tj_uid','tj_cell','tj_prov','tj_city','tj_area','tj_area_work','tj_area_service',
                'tb_name','tb_id_type','tb_id','tb_email','tb_cell',
                'relation','bb_name','bb_id_type','bb_id']
en_short_col.append('date_pay')
# order_sum = pd.concat([order_life_day[en_short_col],order_short_day[en_short_col]])
order_sum = pd.concat([order_2020[en_short_col],order_life_day[en_short_col],order_short_day[en_short_col]])
# del order_2019
# order_sum = pd.concat([order_2019[en_short_col],order_life_day[en_short_col]])
# order_sum = pd.concat([order_life_day[en_short_col],order_short_day[en_short_col]])

In [9]:
order_sum.groupby('date_pay').bf.sum().reset_index().tail()

,date_pay,bf
146,2020-05-20,1874182.58
147,2020-05-21,1415569.52
148,2020-05-22,1427480.14
149,2020-05-23,1137187.66
150,2020-05-24,1094152.31


In [10]:
## 修改-日期
order_sum.to_csv('D:\\User\\Desktop\\order_info\\order_detail_2020_0524.csv',encoding='utf-8',index=0)